In [ ]:
import sklearn as skl
import numpy as np
import pandas as pd
from typing import Tuple, List, TypeVar, Dict, Any, Union

### Part A

In [2]:
# HERE YOU WILL WRITE CODE TO TEST A NUMBER OF PREDICTORS
# AND FINALLY CHOOSE AND TRAIN THE PREDICTOR THAT YOU WILL BE USING FOR PART B

## Initially prodding the data to find out about it

Once again, the lack of any feature names is particularly unhelpful.

In [3]:
_all_data: pd.DataFrame = pd.read_csv("CE802_P3_Data.csv")
_all_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F28,F29,F30,F31,F32,F33,F34,F35,F36,Target
0,271.78,-21.92,UK,-10.80,1899.57,17.98,-21749.82,91.94,855.61,10.01,...,2098.80,500.16,-6325.53,1.36,-33.14,177.34,-141.97,101.52,5,189.03
1,202.54,43.82,USA,-16.94,1941.57,-9.16,-27668.04,93.50,975.44,7.29,...,1668.70,434.97,-6172.05,2.59,-58.87,87.48,-154.11,623.22,6,187.17
2,220.26,88.90,Europe,-18.76,2298.12,-18.38,-11548.56,65.16,1114.28,12.05,...,2604.56,252.93,-10132.68,2.94,-40.89,271.00,-279.84,284.96,2,1016.24
3,141.00,140.72,Europe,-19.86,-133.32,-57.00,-16200.96,-14.00,910.12,4.54,...,2595.56,154.83,-7862.04,0.86,-117.03,201.66,-153.93,532.19,4,-141.18
4,165.04,2.74,Europe,-21.34,3077.07,-20.50,-25683.06,29.08,216.24,10.10,...,1066.80,316.68,-6093.81,3.59,-63.84,211.82,-182.34,373.14,5,33.17


In [4]:
_all_data.dtypes

F1        float64
F2        float64
F3         object
F4        float64
F5        float64
F6        float64
F7        float64
F8        float64
F9        float64
F10       float64
F11        object
F12       float64
F13       float64
F14       float64
F15       float64
F16       float64
F17       float64
F18       float64
F19       float64
F20       float64
F21       float64
F22       float64
F23         int64
F24       float64
F25       float64
F26       float64
F27       float64
F28       float64
F29       float64
F30       float64
F31       float64
F32       float64
F33       float64
F34       float64
F35       float64
F36         int64
Target    float64
dtype: object

In [5]:
_all_data_inputs: pd.DataFrame = _all_data.loc[:,_all_data.columns!="Target"]
_all_data_targets:pd.DataFrame = _all_data.loc[:,"Target"]

It looks like I'll need to convert F3 and F11 into a less awkward format.
Of course, there's the simple approach of doing one-hot encoding, but...

In [6]:
print(_all_data_inputs["F3"].unique())

print(_all_data_inputs["F11"].unique())

['UK' 'USA' 'Europe' 'Rest']
['Very low' 'Medium' 'Low' 'Very high' 'High']


As F11 indicates values with some sequential nature to them
(very low->low->medium->high->very high), I could get away with converting them
to ints. Which is what I am going to do.

However, for F3, I should probably just replace them with some one-hot encoding instead.


In [7]:
from pandas.api.types import CategoricalDtype

f11categories = CategoricalDtype(categories=["Very low","Low","Medium","High","Very high"], ordered=True)

f3categories = CategoricalDtype(categories=["Europe","Rest","UK","USA"])

def df_f11_and_f3_to_sensible(df: pd.DataFrame) -> pd.DataFrame:

    f11dict = { "F11": {
        "Very low": 0,
        "Low": 1,
        "Medium": 2,
        "High": 3,
        "Very high": 4
    }}

    df.replace(f11dict, inplace=True)

    df = pd.get_dummies(df, columns=["F3"])

    return df

all_data = df_f11_and_f3_to_sensible(pd.read_csv("CE802_P3_Data.csv"))

_all_data_inputs = df_f11_and_f3_to_sensible(_all_data_inputs)

c:\python3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
print(_all_data_inputs.dtypes)
print("")
_all_data_inputs.head()

F1           float64
F2           float64
F4           float64
F5           float64
F6           float64
F7           float64
F8           float64
F9           float64
F10          float64
F11            int64
F12          float64
F13          float64
F14          float64
F15          float64
F16          float64
F17          float64
F18          float64
F19          float64
F20          float64
F21          float64
F22          float64
F23            int64
F24          float64
F25          float64
F26          float64
F27          float64
F28          float64
F29          float64
F30          float64
F31          float64
F32          float64
F33          float64
F34          float64
F35          float64
F36            int64
F3_Europe      uint8
F3_Rest        uint8
F3_UK          uint8
F3_USA         uint8
dtype: object



,F1,F2,F4,F5,F6,F7,F8,F9,F10,F11,...,F31,F32,F33,F34,F35,F36,F3_Europe,F3_Rest,F3_UK,F3_USA
0,271.78,-21.92,-10.80,1899.57,17.98,-21749.82,91.94,855.61,10.01,0,...,1.36,-33.14,177.34,-141.97,101.52,5,0,0,1,0
1,202.54,43.82,-16.94,1941.57,-9.16,-27668.04,93.50,975.44,7.29,2,...,2.59,-58.87,87.48,-154.11,623.22,6,0,0,0,1
2,220.26,88.90,-18.76,2298.12,-18.38,-11548.56,65.16,1114.28,12.05,2,...,2.94,-40.89,271.00,-279.84,284.96,2,1,0,0,0
3,141.00,140.72,-19.86,-133.32,-57.00,-16200.96,-14.00,910.12,4.54,1,...,0.86,-117.03,201.66,-153.93,532.19,4,1,0,0,0
4,165.04,2.74,-21.34,3077.07,-20.50,-25683.06,29.08,216.24,10.10,0,...,3.59,-63.84,211.82,-182.34,373.14,5,1,0,0,0


Much better.

Anyway, here's a helper function that'll be useful later on,
pretty much wrapping that dataframe reading/splitting/reformatting into a nice
and simple function for ease of use later on.

In [9]:
def np_data_and_targets(df: pd.DataFrame = pd.read_csv("CE802_P3_Data.csv"), already_formatted_f11_f3: bool = False, targetname: str = "Target") -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts dataframe into a couple of numpy ndarrays for the data without the labels,
    and the labels by themselves.
    Also handles the f11 and f3 formatting.
    :param df: the Dataframe
    :param already_formatted_f11_f3: set this to true if we've already replaced f11 and f3 with sensible formatting.
    :param targetname: The name of the column holding the targets
    :return: tuple of [ndarray of the values without the targets, just the class labels]
    """

    inputs:  np.ndarray = df.loc[:,df.columns != targetname].to_numpy() if already_formatted_f11_f3 else df_f11_and_f3_to_sensible(df.loc[:,df.columns != targetname]).to_numpy()
    outputs: np.ndarray =  df.loc[:,targetname].to_numpy()

    return inputs, outputs

# Setting up a hold-out validation set for later

In [10]:
from sklearn.model_selection import KFold

validation_cv: KFold = KFold(n_splits=5, shuffle=True)
all_train_indices, validation_indices = [i for i in validation_cv.split(_all_data_inputs, _all_data_targets)][0]

validation_df: pd.DataFrame = all_data.iloc[validation_indices]
tests: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(validation_df, True)
test_data: np.ndarray = tests[0]
test_labels: np.ndarray = tests[1]

train_df: pd.DataFrame = all_data.iloc[all_train_indices]
trains: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(train_df, True)
train_data: np.ndarray = trains[0]
train_labels: np.ndarray = trains[1]

print("validation")
print(validation_df.head())
print(validation_df.shape)
print("train")
print(train_df.head())
print(train_df.shape)


validation
        F1      F2     F4       F5     F6        F7      F8      F9    F10  \
4   165.04    2.74 -21.34  3077.07 -20.50 -25683.06   29.08  216.24  10.10   
6   166.76  -31.92 -27.18  1580.46 -58.42 -27216.24   31.98  468.47  11.86   
8   200.42 -239.82 -19.46  2030.43 -12.30 -27179.04   65.72  203.12   1.25   
17  138.02   19.76 -25.06  1001.61 -55.62 -24831.81   61.38  922.19   3.38   
23  198.28   75.28  -7.04  2222.43  17.38 -12962.31  142.00  639.36   8.79   

    F11  ...    F32     F33     F34     F35  F36   Target  F3_Europe  F3_Rest  \
4     0  ... -63.84  211.82 -182.34  373.14    5    33.17          1        0   
6     2  ...   1.74  209.94 -161.74  809.57    2   339.31          0        1   
8     4  ... -26.61  232.64 -196.92  825.80    3  1175.20          0        0   
17    2  ... -54.34  150.76 -176.09  774.41    5  2152.67          0        1   
23    1  ...  16.31  154.02 -256.52  214.32    8  -377.46          0        0   

    F3_UK  F3_USA  
4       0    

# Setting up some grid-search nested cross validation pipelines


In [11]:
from sklearn.base import RegressorMixin
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import PolynomialFeatures

set_config(display="diagram")

R = TypeVar('R', bound=RegressorMixin)


In [12]:
def h_grid_searcher(
        regressor: R,
        param_grid: Dict[str, List[Any]],
        train_data: np.ndarray,
        train_targets: np.ndarray,
        k_fold: KFold = KFold(n_splits=5, shuffle=True, random_state=42)
) -> HalvingGridSearchCV:

    param_grid["imputer__weights"] = ["distance","uniform"]
    param_grid["poly__degree"] = [2,3]

    pl: Pipeline = Pipeline([
        ("imputer",KNNImputer(add_indicator=True)),
        ("poly", PolynomialFeatures(interaction_only=True)),
        ("regressor",regressor)
    ])

    h_grid_search: HalvingGridSearchCV = HalvingGridSearchCV(
        estimator=pl,
        param_grid=param_grid,
        factor=3,
        cv=k_fold,
        scoring=make_scorer(mean_squared_error),
        refit=True,
        verbose=1,
        n_jobs=-1
    )

    h_grid_search.fit(train_data, train_targets)

    return h_grid_search

In [13]:
def nested_h_grid_searcher(
        regressor: R,
        param_grid: Dict[str, List[Any]],
        train_data: np.ndarray,
        train_targets: np.ndarray,
        t_df: pd.DataFrame,
        kfold_splits: int = 6
) -> Dict[HalvingGridSearchCV, float]:

    h_grid_search_dicts: Dict[HalvingGridSearchCV, float] = {}

    kf: KFold = KFold(n_splits=kfold_splits, shuffle=True)

    for train_indices, test_indices in kf.split(train_data, train_targets):
        tr: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(t_df.iloc[train_indices],True)
        te: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(t_df.iloc[test_indices],True)

        current_search: HalvingGridSearchCV = h_grid_searcher(
            regressor,
            param_grid,
            tr[0],
            tr[1],
            KFold(n_splits=kfold_splits-1, shuffle=False)
        )

        current_score: float = current_search.score(te[0],te[1])

        h_grid_search_dicts[current_search] = current_score

    return h_grid_search_dicts

# Linear Regression

There isn't much that I can do in terms of hyper-parameter search for simple linear regression,
but, here goes.

In [ ]:

from sklearn.linear_model import LinearRegression

linear_regression_searched_dict: Dict[
    HalvingGridSearchCV, float
] = nested_h_grid_searcher(
    LinearRegression(),
    {},
    train_data,
    train_labels,
    train_df
)

linreg_searched: HalvingGridSearchCV = min(
    linear_regression_searched_dict.keys(), key=lambda k: linear_regression_searched_dict[k]
)

best_linreg: Tuple[HalvingGridSearchCV, float] = (
    linreg_searched, linear_regression_searched_dict[linreg_searched]
)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 333
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 333
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 1
n_candidates: 2
n_resources: 999
Fitting 5 folds for each of 2 candidates, totalling 10 fits
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 333
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 333
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 1
n_candidates: 2
n_resources: 999
Fitting 5 folds for each of 2 candidates, totalling 10 fits
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 333
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 333
Fitting 5 folds for each of 4 candidates, totalling 20 fits

In [ ]:
print(best_linreg)

print(best_linreg[1])

linreg_searched.best_estimator_

# Random Sample Consensus regression

In [ ]:
from sklearn.linear_model import RANSACRegressor

rs_samples: List[int] = [1,2,3,4,5]
rs_residuals: List[Union[float, None]] = [None, 0.001, 0.01, 0.1, 1]
rs_max_trials: List[int] = [80,90,100,110,120]

rs_param_dict: Dict[str, List[Any]] = {
    "regressor__min_samples": rs_samples,
    "regressor__residual_threshold": rs_residuals,
    "regressor__max_trials": rs_max_trials
}

ransac_regression_searched_dict: Dict[
    HalvingGridSearchCV, float
] = nested_h_grid_searcher(
    RANSACRegressor(loss="squared_error"),
    rs_param_dict,
    train_data,
    train_labels,
    train_df
)

ransac_searched: HalvingGridSearchCV = min(
    ransac_regression_searched_dict.keys(), key=lambda k: ransac_regression_searched_dict[k]
)

best_ransac: Tuple[HalvingGridSearchCV, float] = (
    ransac_searched, ransac_regression_searched_dict[ransac_searched]
)

In [ ]:
print(best_ransac)

print(best_ransac[1])

ransac_searched.best_estimator_

# Stochastic Gradient Descent Regression

### Part B

In [ ]:
# HERE YOU WILL USE THIS TEMPLATE TO SAVE THE PREDICTIONS ON THE TEST SET

# Load the test data
test_df = pd.read_csv('CE802_P3_Test.csv')

final_data, final_targets = np_data_and_targets(pd.read_csv("CE802_P3_Test.csv"))



predicted = linreg_searched.predict(final_data)
# CHANGE HERE -- use your previously trained predictor and apply it to test_data
# (test_data can be modified if needed but make sure you don't change the order of the rows)...

# Replace the last (empty) column with your prediction
test_df.iloc[:,-1] = predicted



# Save to the destination file
test_df.to_csv('CE802_P3_Test_Predictions.csv', index=False, float_format='%.8g')

# IMPORTANT!! Make sure only the last column has changed
assert pd.read_csv('CE802_P3_Test.csv').iloc[:,:-1].equals(pd.read_csv('CE802_P3_Test_Predictions.csv').iloc[:,:-1])